In [1110]:
# Standard library imports
import os
import sys
import re
import warnings
import random
import hashlib

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning and preprocessing
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler  # Assuming you might need it

# Specific models and tools
from xgboost import XGBClassifier
import xgboost as xgb

# Encoding and feature selection
from category_encoders import TargetEncoder  # Fixed the import based on usage
from scipy.stats import randint, uniform

# Model persistence
from joblib import dump, load

# Miscellaneous settings
%matplotlib inline
warnings.filterwarnings('ignore')


In [1111]:
countries = [

    "ARG", "AUT", "BRA", "CHN",
    "DNK", "FIN", "IRL", "JPN",
    "MEX", "NOR", "POL", "ROU",
    "RUS", "SWE", "SWZ", "USA",
]

fixtures = [
    "fixtures_world",
]


In [1112]:
# Set the dataprep_start_date to the date the data preparation should start
# If None, the data preparation will start from the beginning of the data

# Make sure the file below already exists if you want to start from a specific date
# file should be in the format "processed_data_<content>.csv"
content = "world_all"

dataprep_start_date = None
#dataprep_start_date = pd.Timestamp(year=2024, month=4, day=23)

In [1113]:
# Load all filepaths into a list
matches_files = []
fixtures_files = []

In [1114]:
for fixture in fixtures:    
    fixtures_files.append(f'data/fixtures/{fixture}.csv')
    continue

In [1115]:
for country in countries:
    matches_files.append('data/cleaned/%s.csv' % (country))
    continue

In [1116]:
def load_data(files):
    df = pd.DataFrame()

    for file in files:
        try:
            print(f'Loading {file}')

            # Try to read with default utf-8 encoding
            try:
                df_temp = pd.read_csv(file, encoding='utf-8')
            except UnicodeDecodeError:
                # If utf-8 decoding fails, try reading with ISO-8859-1
                df_temp = pd.read_csv(file, encoding='ISO-8859-1')

            #df_temp['Season'] = year
            df = pd.concat([df, df_temp], ignore_index=True)
        except FileNotFoundError:
            print(f'Error: {file} not found')
        except Exception as e:
            print(f"An error occurred while loading {file}: {e}")

    return df


In [1117]:
# Load data into DataFrames
df = load_data(matches_files)
df_fixtures = load_data(fixtures_files)

Loading data/cleaned/ARG.csv
Loading data/cleaned/AUT.csv
Loading data/cleaned/BRA.csv
Loading data/cleaned/CHN.csv
Loading data/cleaned/DNK.csv
Loading data/cleaned/FIN.csv
Loading data/cleaned/IRL.csv
Loading data/cleaned/JPN.csv
Loading data/cleaned/MEX.csv
Loading data/cleaned/NOR.csv
Loading data/cleaned/POL.csv
Loading data/cleaned/ROU.csv
Loading data/cleaned/RUS.csv
Loading data/cleaned/SWE.csv
Loading data/cleaned/SWZ.csv
Loading data/cleaned/USA.csv
Loading data/fixtures/fixtures_world.csv


In [1118]:
len(df), len(df_fixtures)

(48003, 120)

In [1119]:
df_fixtures['Div'].value_counts()

Div
JPN    18
USA    14
BRA     9
POL     9
CHN     8
NOR     8
ROU     8
RUS     8
SWE     8
AUT     6
DNK     6
FIN     6
SWZ     6
IRL     5
ARG     1
Name: count, dtype: int64

In [1120]:
len(df), len(df_fixtures)

(48003, 120)

In [1122]:
if len(df_fixtures) > 0:
    df_fixtures['Date_temp'] = pd.to_datetime(df_fixtures['Date'], format='%d/%m/%Y')

    df_fixtures['Date_temp'] = df_fixtures['Date_temp'].apply(
        lambda x: int(x.strftime('%Y%m%d')) if pd.notnull(x) else 19000101)

    df['Date_fixt'] = pd.to_datetime(df['Date'], format='%d/%m/%Y').apply(
        lambda x: int(x.strftime('%Y%m%d')) if pd.notnull(x) else 19000101)

    df_fixtures['FTR'].fillna('X', inplace=True)
    fixture_cutoff = df_fixtures['Date_temp'].min()

    df = df[df['Date_fixt'] < fixture_cutoff]

    df = pd.concat([df, df_fixtures], ignore_index=True)

    # Drop df Date_fixt column
    df.drop(columns=['Date_fixt'], inplace=True)

In [1125]:
# Remove all the rows in the dataframe where the 'Div' is not in the list of countries
df = df[df['Div'].isin(countries)]

In [1127]:
# Create a dictionary for all competitions

file_path = f"data/comps_dict_{content}.txt"

# Check if the file exists
if os.path.exists(file_path):
    # Load the dictionary from the file
    with open(file_path, 'r') as file:
        comps_dict = eval(file.read())  # Using eval to convert string back to dictionary
    # Find the maximum index currently in the dictionary
    max_index = max(comps_dict.values())

    print(f"max index: {max_index}")
else:
    comps_dict = {}
    max_index = -1  

# Get all unique divisions from DataFrame
all_comps = df['Div'].dropna().unique()
all_comps.sort()

# Create a dictionary of new divisions alone
new_comps = {div: index for index, div in enumerate(all_comps, start=max_index + 1) if div not in comps_dict}

# Update dictionary only with new divisions
comps_dict.update(new_comps)

# Save the updated dictionary to a file
with open(file_path, 'w') as file:
    file.write(str(comps_dict))

# Add division ID column to DataFrame
df['Div'] = df['Div'].map(comps_dict)

max index: 15


In [1128]:
# Create a dictionary for all teams

file_path = f"data/teams_dict_{content}.txt"

# Check if the file exists
if os.path.exists(file_path):
    # Load the dictionary from the file
    with open(file_path, 'r') as file:
        teams_dict = eval(file.read())  
    max_index = max(teams_dict.values())

    print(f"max index: {max_index}")
else:
    teams_dict = {}
    max_index = -1 

# Get all teams from DataFrame
all_teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).dropna().unique()
all_teams.sort()

# Create a dictionary of new teams alone
new_teams = {team: index for index, team in enumerate(all_teams) if team not in teams_dict}

# Update dictionary only with new teams, starting indices from max_index + 1
start_index = max_index + 1
teams_dict.update({team: index + start_index for index, team in enumerate(new_teams) if team not in teams_dict})

# Save the updated dictionary to a file
with open(file_path, 'w') as file:
    file.write(str(teams_dict))

# Add team ID columns to DataFrame
df['Team_ID'] = df['HomeTeam'].map(teams_dict)
df['Opp_ID'] = df['AwayTeam'].map(teams_dict)

max index: 480


In [1129]:
def clean_duplicates(df):
    # Sort the DataFrame so that rows with 'FTR' == -1 come first
    df.sort_values(by=['Date', 'Team_ID', 'Opp_ID', 'FTR'], ascending=[True, True, True, False], inplace=True)
    
    # Drop duplicates based on 'Date', 'Team_ID', and 'Opp_ID' keeping the first occurrence (where 'FTR' is -1)
    df = df.drop_duplicates(subset=['Date', 'Team_ID', 'Opp_ID'], keep='first')
    
    return df

In [1130]:
df = clean_duplicates(df)

In [1132]:
df.shape

(48123, 15)

### Feature Engineering

In [1133]:
# Calculate ELO ratings for each team

# Initialize ratings dictionary
teams = pd.concat([df['Team_ID'], df['Opp_ID']]).unique()
ratings = {team: 1500 for team in teams}

def calculate_expected_score(rating_a, rating_b):
    return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

def update_elo(rating, actual_score, expected_score, k=30):

    rating = rating + k * (actual_score - expected_score)

    # Parse the rating as an integer with no decimal points
    return int(rating)

# Iterate over the DataFrame and update ELO ratings after each match
elo_team = []
elo_opp = []

for index, row in df.iterrows():
    home_team, away_team, home_score, away_score = row['Team_ID'], row['Opp_ID'], row['FTHG'], row['FTAG']
    home_rating = ratings[home_team]
    away_rating = ratings[away_team]
    
    # Calculate expected scores
    expected_home = calculate_expected_score(home_rating, away_rating)
    expected_away = calculate_expected_score(away_rating, home_rating)
    
    # Calculate actual scores
    actual_home = 1 if home_score > away_score else 0.5 if home_score == away_score else 0
    actual_away = 1 - actual_home
    
    # Update ratings
    new_home_rating = update_elo(home_rating, actual_home, expected_home)
    new_away_rating = update_elo(away_rating, actual_away, expected_away)
    
    # Store updated ratings in the ratings dictionary
    ratings[home_team] = new_home_rating
    ratings[away_team] = new_away_rating
    
    # Append current ratings to list
    elo_team.append(new_home_rating)
    elo_opp.append(new_away_rating)


In [1134]:
# Assign new ELO ratings to the DataFrame
df['team_elo'] = elo_team
df['opp_elo'] = elo_opp

In [1135]:
def parse_date_to_int(date_str):

    # Convert date_str to string if it is not already
    date_str = str(date_str)

    # Split the date_str by the "/" character into day, month, year
    components = date_str.split('/')
    
    # If split was successful but not in expected format, try splitting by absence of separator for '%d%m%Y' or '%d%m%y'
    if len(components) == 1:
        if len(date_str) in [6, 8]:  # Length 6 for '%d%m%y', 8 for '%d%m%Y'
            day, month = int(date_str[:2]), int(date_str[2:4])
            year = int(date_str[4:])
        else:
            return 19000101  # Return default if format does not match expected
    else:

        day = int(components[0])
        month = int(components[1])
        year = int(components[2])
    
    # Adjust the year if it was only 2 characters long
    if year < 100:
        year += 2000
    
    # Create a date variable by using the day, month, year integers
    # Note: Direct creation of date variable skipped to avoid unnecessary complexity,
    # directly formatting to YYYYMMDD integer format instead.
    date_int = int(f"{year:04d}{month:02d}{day:02d}")
    
    return date_int

In [1136]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Apply the modified function
df['Date_temp'] = df['Date'].apply(lambda x: parse_date_to_int(x.strftime('%d/%m/%Y')) if pd.notnull(x) else 19000101)

# Day of the week as an integer
df['DayOTW'] = df['Date'].dt.dayofweek

df['Time'] = df['Time'].fillna('00:00').str.replace(':', '').astype(int)

# Only keep the first 2 digits of the Time column, no decimals
df['Time'] = df['Time'] // 100

# Sort df by Date_temp and Time
df = df.sort_values(['Date_temp', 'Time'])

In [1137]:
df.columns = [re.sub(r'[<]', '_st_', str(col)) for col in df.columns]
df.columns = [re.sub(r'[>]', '_gt_', str(col)) for col in df.columns]

In [1138]:
def points(df, row, team_column, last_n_matches=None):
    # Season and date of the current match
    current_season = row['Season']
    current_date = row['Date']

    # Define the opponent column based on the team column
    opponent_column = 'Opp_ID' if team_column == 'Team_ID' else 'Team_ID'

    # Filter DataFrame for matches from the same season before the current date
    past_matches = df[
        (df['Season'] == current_season) & 
        (df['Date'] < current_date) &
        ((df[team_column] == row[team_column]) | (df[opponent_column] == row[team_column]))
    ].copy()

    # If last_n_matches is set, filter to the last n matches
    if last_n_matches is not None:
        past_matches = past_matches.tail(last_n_matches)

    # Initialize total points
    total_points = 0

    # Calculate points for each past match
    for match in past_matches.itertuples():
        if getattr(match, 'Team_ID') == row[team_column]:
            if getattr(match, 'FTR') == 'H':
                total_points += 3  # Home win
            elif getattr(match, 'FTR') == 'D':
                total_points += 1  # Draw
        elif getattr(match, 'Opp_ID') == row[team_column]:
            if getattr(match, 'FTR') == 'A':
                total_points += 3  # Away win
            elif getattr(match, 'FTR') == 'D':
                total_points += 1  # Draw

    # Calculate average points
    matches_played = len(past_matches)
    avg_points = total_points / matches_played if matches_played > 0 else 0

    # Return average points rounded to 3 decimal places
    return round(avg_points, 3)

In [1139]:
df['team_points'] = df.apply(lambda x: points(df, x, 'Team_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)
df['opp_points'] = df.apply(lambda x: points(df, x, 'Opp_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)

df['team_form'] = df.apply(lambda x: points(df, x, 'Team_ID', last_n_matches=5) 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)
df['opp_form'] = df.apply(lambda x: points(df, x, 'Opp_ID', last_n_matches=5) 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)

In [1140]:
def history_vs_opponent_weighted(df, row, team_column):
    # Determine opponent column based on team column
    opponent_column = 'Team_ID' if team_column == 'Opp_ID' else 'Opp_ID'

    # Combine year, month, and day into an integer 'Date_temp'
    row_date_temp = row['Date'].year * 10000 + row['Date'].month * 100 + row['Date'].day

    # Filter for matches between specified teams, excluding current match
    mask = (
        ((df[team_column] == row[team_column]) & (df[opponent_column] == row[opponent_column])) |
        ((df[team_column] == row[opponent_column]) & (df[opponent_column] == row[team_column]))
    ) & (df['Date_temp'] < row_date_temp)

    filtered_matches = df[mask]
    
    if filtered_matches.empty:
        return 0  # Return early if no matches found

    # Sort by date and select top 5 recent matches
    recent_matches = filtered_matches.sort_values(by='Date', ascending=False).head(5)
    weights = list(range(len(recent_matches), 0, -1))  # Descending weights

    # Calculate weighted score based on match results
    weighted_score = sum(
        (3 * weight if match.FTR == 'H' and match.__getattribute__(team_column) == match.Team_ID or
                      match.FTR == 'A' and match.__getattribute__(team_column) != match.Team_ID else
         1 * weight if match.FTR == 'D' else 0)
        for match, weight in zip(recent_matches.itertuples(), weights)
    )

    # Normalize the weighted score by the sum of weights
    return round(weighted_score / sum(weights), 3) if weights else 0

In [1141]:
df['team_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Team_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)
df['opp_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Opp_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else None, axis=1)

In [1142]:
def avg_goals(df, row, team_column):
    # Season and date of the current match
    current_season = row['Season']
    current_date = row['Date']

    # Determine the columns for goals scored and conceded based on perspective
    if team_column == 'Team_ID':
        goals_scored_column = 'FTHG'  # Assuming FTHG is the column for home team goals
        goals_conceded_column = 'FTAG'  # Assuming FTAG is the column for away team goals
    else:
        goals_scored_column = 'FTAG'  # Flip the columns if we are looking from the opponent's perspective
        goals_conceded_column = 'FTHG'

    # Filter matches from the same season and before the current date
    past_matches = df[
        (df['Season'] == current_season) & 
        (df['Date'] < current_date) & 
        ((df['Team_ID'] == row[team_column]) | (df['Opp_ID'] == row[team_column]))
    ]

    # Calculate the average goals scored and conceded
    goals_scored = 0
    goals_conceded = 0
    total_matches = len(past_matches)

    for match in past_matches.itertuples():
        if getattr(match, 'Team_ID') == row[team_column]: 
            goals_scored += getattr(match, goals_scored_column)
            goals_conceded += getattr(match, goals_conceded_column)
        else:  # Team is playing away
            goals_scored += getattr(match, goals_scored_column)
            goals_conceded += getattr(match, goals_conceded_column)

    avg_goals_for = goals_scored / total_matches if total_matches > 0 else 0
    avg_goals_against = goals_conceded / total_matches if total_matches > 0 else 0

    avg_goals_for = round(avg_goals_for, 2)
    avg_goals_against = round(avg_goals_against, 2)

    return avg_goals_for, avg_goals_against

In [1143]:
# Apply the function and create new columns
df['team_avg_goals_for'], df['team_avg_goals_against'] = zip(*df.apply(lambda x: avg_goals(df, x, 'Team_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else (0, 0), axis=1))
df['opp_avg_goals_for'], df['opp_avg_goals_against'] = zip(*df.apply(lambda x: avg_goals(df, x, 'Opp_ID') 
    if dataprep_start_date is None or x['Date'] >= dataprep_start_date else (0, 0), axis=1))

In [1144]:
# Calculate means only for numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
means = df[numeric_cols].mean()

# Fill missing values in numeric columns with their respective means
df[numeric_cols] = df[numeric_cols].fillna(means)

In [1145]:
# Set the FTR to 'X' where the value is currently NaN
df['FTR'] = df['FTR'].fillna('X')

In [1146]:
# Drop every row where 'FTR' is not 'H', 'D', or 'A', or 'X' (if future matches are included)
df = df[df['FTR'].isin(['H', 'D', 'A', 'X'])]

# Map 'H', 'D', and 'A' to 0, 1, and 2 respectively
df['FTR'] = df['FTR'].map({'H': 0, 'D': 1, 'A': 2, 'X': -1}).astype(int)

In [1147]:
df.shape

(48123, 28)

In [1148]:
df = df[[
    
        'Div', 'Season', 'Date_temp', 'Time', 'DayOTW', 'Team_ID', 'Opp_ID', 'FTR',

        'team_elo', 'opp_elo',
        
        'team_hist_vs', 
        'opp_hist_vs',

        'team_points',
        'opp_points',
        
        'team_form', 
        'opp_form',

        'team_avg_goals_for', 
        'team_avg_goals_against',
        'opp_avg_goals_for',
        'opp_avg_goals_against',

        'AvgH', 'AvgD', 'AvgA'         
         
]]

In [1149]:
# Print the value counts of the Date_temp column where FTR is -1
print(df[df['FTR'] == -1]['Date_temp'].value_counts())

Date_temp
20240505    58
20240504    23
20240503    21
20240506    18
20161211     1
Name: count, dtype: int64


In [1150]:
len(df), len(df_fixtures)

(48123, 120)

In [1151]:
# Rename 'Date_temp' to 'Date'
df.rename(columns={'Date_temp': 'Date'}, inplace=True)

In [1152]:
# Drop duplicate rows based on 'Date', 'Team_ID', and 'Opp_ID'
df = df.drop_duplicates(subset=['Date', 'Team_ID', 'Opp_ID'], keep='first')

In [1153]:
import pandas as pd

try:
   
    if dataprep_start_date is not None:
        # Convert date columns to datetime
        df['Date_temp'] = pd.to_datetime(df['Date'], format='%Y%m%d')
        
        # Filter new data based on start date
        df_new = df[df['Date_temp'] >= dataprep_start_date].copy()

        # Load existing data
        df_existing = pd.read_csv(f'data/processed/processed_data_{content}.csv')

        df_existing['Date_temp'] = pd.to_datetime(df_existing['Date'])
        
        # Filter existing data to remove overlap with new data
        df_existing = df_existing[df_existing['Date_temp'] < dataprep_start_date]

        # Combine and sort data
        df_final = pd.concat([df_existing, df_new], ignore_index=True)
        df_final.sort_values(['Date_temp', 'Time'], inplace=True)

        # Clean up temporary columns
        df_final.drop(columns='Date_temp', inplace=True)
    else:
        df_final = df.copy()

    # Save the final DataFrame
    df_final.to_csv(f'data/processed/processed_data_{content}.csv', index=False)
    print(f"Data saved: {df_final.shape[0]} matches")

except Exception as e:
    print(f"Error: {e}")


Data saved: 48123 matches


In [1154]:
import winsound
frequency = 400  # Set Frequency To 2500 Hertz
duration = 200  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)